In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
import warnings
warnings.filterwarnings("ignore")

In [2]:
file_in = Doc_Chx_file
data_doc = pd.read_csv(r'E:\_Projects\_outputs\orders\snd\\' + file_in + '.csv', low_memory=False, keep_default_na='', encoding='utf-8')
file_in = file_in[0:30]
print(file_in)
#iput()

workbench_data_eumocp_20240123


In [ ]:
#file_in = 'workbench_data_eumocp_20240123_v2'
#data_doc = pd.read_csv(r'E:\_Projects\_outputs\orders\snd\\' + file_in + '.csv', low_memory=False, keep_default_na='', encoding='utf-8')
#file_in = file_in[0:30]
#print(file_in)


In [3]:
col_data = list(data_doc.columns)
col_data = col_data.copy()
today_date = file_in[22:30]

In [4]:
def upd_period(df, fld, fld_wk, fld_mon, fld_qtr):
    df[fld_wk] = pd.to_datetime(df[fld], errors='coerce').dt.isocalendar().week.astype(pd.Int64Dtype())
    df[fld_mon] = pd.to_datetime(df[fld], errors='coerce').dt.strftime('%Y%m')
    df[fld_qtr] = pd.to_datetime(df[fld], errors='coerce').dt.strftime('%Y') + 'Q' + \
                        + pd.to_datetime(data_doc[fld], errors='coerce').dt.quarter.astype(pd.Int64Dtype()).astype(str)

In [5]:
# Create PCD; Update past into current
data_doc['Today'] = pd.to_datetime('today').date()
# data_doc['PCD'] = pd.to_datetime(data_doc['Plant Ship date'], format='%Y-%m-%d', errors='coerce') - timedelta(days=7)
data_doc['PCD'] = pd.to_datetime(data_doc['Plant Ship date'], format='%Y-%m-%d', errors='coerce')


data_doc['PCD'] = np.where(pd.to_datetime(data_doc['PCD']) < pd.to_datetime(data_doc['Today']), \
                             data_doc['Today'].astype('datetime64[ns]'), data_doc['PCD'])

upd_period(data_doc, 'PCD', 'PCD Wk', 'PCD Mon', 'PCD Qtr')
data_doc.drop(columns={'Today'}, axis=1, inplace=True)

In [6]:
data_chx = data_doc[(data_doc['Whse'].isin(['CHE','CHS'])) & (data_doc['Validation Flag'] != 'Plan')]
print(data_chx.shape)
cond1 = ['CHE Flow Stock IN','CHS Flow Stock IN']
cond2 = ['CHE Interplant IN','CHS Interplant IN','CHE MFG Gratis IN', 'CHS MFG Gratis IN']

conditions = [data_chx['Order Type'].isin(cond1), data_chx['Order Type'].isin(cond2)]
choices = ['Ord Flow Stk', 'Order']
data_chx['Order Type_chx'] = np.select(conditions, choices, 'Int Order')
# data_chx['Order Type_chx'] = np.where(data_chx['Order Type_chx'].isnull() | data_chx['Order Type_chx'] == '0', 'Int Order', data_chx['Order Type_chx'])

(5843, 146)


In [7]:
data_chx['IO BOOKED DATE'] = np.where(data_chx['IO BOOKED DATE']=='', data_chx['Booked Dt'], data_chx['IO BOOKED DATE'])
# data_chx.loc[data_chx['IO BOOKED DATE']=='', 'IO BOOKED DATE'] = data_chx['Booked Dt']

In [8]:
data_chx['IO No.'] = data_chx['IO No.'].astype(str).replace('nan', np.nan)
data_chx['IO No.'].fillna('', inplace=True)
data_chx['IO No.'] = np.where(data_chx['IO No.']=='', data_chx['DEMAND VAL'], data_chx['IO No.'])

In [9]:
# Data for WB Tool
col_add = ['File_','SO_','Booked Dt_','PID_','Qty_','PO_','Ship Dt_','Ship Qtr_','Ship Mon_','Ship Wk_','PD_',\
           'Request Dt_','Tot Qty_','Org PD_','Critical_','Revenue_','Priority_','Comments-1_','Comments-2_','Customer_']


data_tmp = pd.DataFrame(columns=col_add)
#lst = ['CHE','CHS']
data_chx_allo = pd.concat([data_tmp, data_chx])
data_chx_allo['File_'] = 'CHX'
data_chx_allo['PO_'] = data_chx_allo['IO No.']
data_chx_allo['Booked Dt_'] = data_chx_allo['IO BOOKED DATE']
data_chx_allo['PID_'] = data_chx_allo['PID']
data_chx_allo['Qty_'] = data_chx_allo['Qty']
data_chx_allo['SO_'] = data_chx_allo['DEMAND VAL']
data_chx_allo['Ship Dt_'] = data_chx_allo['PCD']
data_chx_allo['Ship Qtr_'] = data_chx_allo['PCD Qtr']
data_chx_allo['Ship Mon_'] = data_chx_allo['PCD Mon']
data_chx_allo['Ship Wk_'] = data_chx_allo['PCD Wk']
data_chx_allo['PD_'] = ''
data_chx_allo['Request Dt'] = ''
data_chx_allo['Plant'] = 'CHE'
data_chx_allo['Order Type']=data_chx_allo['Order Type_chx']
data_chx_allo = data_chx_allo[(col_add + col_data)]



#data_chx_allo.to_csv(r'\\dayorg1\orgshare\TEAMS\ERP Shared Folder\Global S&OP\workbench\allocation\wb_chx_allocation_' + \
            #today_date + '.csv', index=False)
data_chx_allo.to_csv(r'E:\_Projects\_outputs\orders\snd\wb_chx_allocation_' + today_date + '.csv', index=False)     
# data_chx_allo.to_csv(r'\\Dayorg1\ORGSHARE\TEAMS\ERP Shared Folder\Global S&OP\Supply Plan\Production\workbench\snd\wb_chx_allocation_' + today_date + '.zip', compression={'method': 'zip', 'archive_name': 'workbench_po.csv'},index=False)


In [10]:
col_chx = ['Products','Whse','Region','Demand','Org Name','Customer_End Customer', 'IO BOOKED DATE', 'Class', 'DEMAND VAL', \
           'Plant Ship date', 'PCD','PCD Wk','PCD Mon', 'PCD Qtr', 'PID','Supply Type', 'Qty', 'Validation Flag', \
           'Order Type','IO No.', 'Seiban Number', 'Customer PO','CDP','Order Type_chx','ERP Line Status']
data_chx = data_chx[col_chx]

In [11]:
# data_chx.to_csv('E:/_outputs/orders/snd/' + file_in + '_chx.csv', index=False)
# data_chx.to_csv(r'\\dayorg1\orgshare\TEAMS\ERP Shared Folder\Global S&OP\workbench\region\\' + file_in + '_chx.csv', index=False)

In [12]:
data_chx.to_csv('E:\_Projects\_outputs\orders\snd\\' + file_in + '_chx.csv', index=False)
try:
    data_chx.to_csv(r'\\wtc1501cifs.prod.local\ORGSHARE\TEAMS\ERP Shared Folder\NCR Atleos\Atleos Operations Planning\Global Ops Analytics\Metrics\Workbench\Orders\\' + file_in + '_chx.csv', index=False)
except:
    print('Server location not working')